In [11]:
!pip install geopandas rasterio folium contextily -q

In [12]:
!pip install earthengine-api geemap -q


In [13]:
import geopandas as gpd
import rasterio
from rasterio.plot import show
import matplotlib.pyplot as plt
import folium
import numpy as np


In [14]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()


EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

In [ ]:
study_area = gpd.read_file("study_area.shp")
stations = gpd.read_file("stations.shp")

study_area.head()


In [ ]:
study_area.crs
stations.crs


In [ ]:
raster = rasterio.open("image.tif")
raster.meta


In [ ]:
raster.count
raster.crs
raster.res


In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
study_area.plot(ax=ax, facecolor="none", edgecolor="black")
stations.plot(ax=ax, color="red", markersize=30)
plt.title("Study Area and Stations")
plt.show()


In [ ]:
m = folium.Map(location=[
    stations.geometry.y.mean(),
    stations.geometry.x.mean()
], zoom_start=8)

for _, row in stations.iterrows():
    folium.Marker(
        location=[row.geometry.y, row.geometry.x],
        popup="Station"
    ).add_to(m)

m


In [ ]:
band1 = raster.read(1)

plt.imshow(band1, cmap="gray")
plt.colorbar()
plt.title("Band 1")
plt.show()


In [ ]:
red = raster.read(4).astype(float)
nir = raster.read(5).astype(float)

ndvi = (nir - red) / (nir + red)


In [ ]:
plt.imshow(ndvi, cmap="RdYlGn")
plt.colorbar(label="NDVI")
plt.title("NDVI Map")
plt.show()


In [ ]:
from rasterio.mask import mask
from shapely.geometry import mapping

geoms = [mapping(study_area.geometry.iloc[0])]

clipped, transform = mask(raster, geoms, crop=True)


In [ ]:
ndvi_clip = clipped[0]

mean_ndvi = np.nanmean(ndvi_clip)
mean_ndvi


In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
show(ndvi_clip, ax=ax, cmap="RdYlGn")
study_area.boundary.plot(ax=ax, color="black")
plt.title("NDVI with Study Area Boundary")
plt.show()


https://data.go.th/dataset/dataset_67_18